In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pprint import pprint

Example way to query dbpedia using sparql wrapper :

In [6]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>

    SELECT ?res (MAX(?pop) as ?pop)
    WHERE {<http://dbpedia.org/resource/New_York_(state)> rdfs:comment ?res.
            <http://dbpedia.org/resource/New_York_(state)> dbp:pop ?pop.
            FILTER(LANG(?res)="en")
    }
"""
)

sparql.setReturnFormat(JSON)

result = sparql.query().convert()


This kinda gives idea of JSON output :

In [7]:
pprint(result)

{'head': {'link': [], 'vars': ['res', 'pop']},
 'results': {'bindings': [{'pop': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                   'type': 'typed-literal',
                                   'value': '8804190'},
                           'res': {'type': 'literal',
                                   'value': 'New York is a state in the '
                                            'Northeastern United States. It '
                                            'was one of the original thirteen '
                                            'colonies forming the United '
                                            'States. With a total area of '
                                            '54,556 square miles (141,300 '
                                            'km2), New York is the 27th '
                                            'largest state geographically. Its '
                                            'population of more than 20 '
    

The below one is the wrong way since i realised later on that arrays of dict are returned for multiple rows (ignore this)

In [19]:
res_dict = result['results']['bindings']
for res_var in res_dict[0].keys():
    print(res_var," : ",res_dict[0][res_var]['value'])

res  :  New York is a state in the Northeastern United States. It was one of the original thirteen colonies forming the United States. With a total area of 54,556 square miles (141,300 km2), New York is the 27th largest state geographically. Its population of more than 20 million people makes it the fourth most populous state in the United States as of 2020. The state is bordered by New Jersey and Pennsylvania to the south, and Connecticut, Massachusetts, and Vermont to the east; it has a maritime border with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the northwest. It is sometimes called New York State to distinguish it from its largest city, New York City.
pop  :  8804190


Now trying with our ontology by hosting the ontology onto a local graphDB server

In [2]:
sparql2 = SPARQLWrapper("http://sunnysoni-OMEN-Laptop:7200/repositories/test_1") #address of graphDB sparql endpoint locally

qry = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX ncbi: <http://purl.obolibrary.org/obo/NCBITaxon_>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
select ?obs_id ?lat ?long ?ncbi_taxon_name from <http://example.com/monkey_onto> where { 
	?obs_id rdf:type ?ncbi_taxon;
         dwc:decimalLatitude ?lat;
         dwc:decimalLongitude ?long.
    ?ncbi_taxon oboInOwl:hasExactSynonym ?ncbi_taxon_name.
    FILTER(?ncbi_taxon != dwc:Occurrence)
} ORDER BY (?ncbi_taxon_name)
limit 1000

"""

sparql2.setQuery(qry)

sparql2.setReturnFormat(JSON)

result2 = sparql2.query().convert()


And this is pretty nice how it works (using extra iterator ofcourse for each row) :

In [3]:
res_dict2 = result2['results']['bindings']
for i in range(len(res_dict2)):
    print("--- Observation ",(i+1),"---")
    for res_var in res_dict2[i].keys():
        print(res_var," : ",res_dict2[i][res_var]['value'])
    

--- Observation  1 ---
obs_id  :  http://www.inaturalist.org/observations/1109557
lat  :  8.230783
long  :  34.600029
ncbi_taxon_name  :  African green monkey
--- Observation  2 ---
obs_id  :  http://www.inaturalist.org/observations/1301642
lat  :  7.050374
long  :  38.495504
ncbi_taxon_name  :  African green monkey
--- Observation  3 ---
obs_id  :  http://www.inaturalist.org/observations/1708052
lat  :  11.623401
long  :  37.380638
ncbi_taxon_name  :  African green monkey
--- Observation  4 ---
obs_id  :  http://www.inaturalist.org/observations/1710401
lat  :  7.023123
long  :  38.46631
ncbi_taxon_name  :  African green monkey
--- Observation  5 ---
obs_id  :  http://www.inaturalist.org/observations/2813096
lat  :  15.601796
long  :  32.50936
ncbi_taxon_name  :  African green monkey
--- Observation  6 ---
obs_id  :  http://www.inaturalist.org/observations/299125
lat  :  7.040859
long  :  38.464336
ncbi_taxon_name  :  African green monkey
--- Observation  7 ---
obs_id  :  http://www.in